In [38]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import pandas as pd

cap = DesiredCapabilities().FIREFOX
cap["marionette"] = False

## NOTES

* Download latest geckodriver: https://github.com/mozilla/geckodriver/releases/tag/v0.24.0
* Unzip it and put geckodriver under python.exe location
  * Win button, then type "python.exe", right click and choose "find file location"

In [45]:
binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options = Options()
options.headless = True
options.binary = binary
cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True #optional
driver = webdriver.Firefox(options=options, capabilities=cap)

In [46]:
def get_company_dct(driver, url_pre, pages):
    company_dct = {}
    
    for i in range(pages):
        url = url_pre + str(i*10)
        driver.get(url)
        time.sleep(3)
        
        company_lst = driver.find_elements_by_class_name('company')
        title_lst = driver.find_elements_by_class_name('title')
        
        for j in range(len(company_lst)):
            company = company_lst[j].text
            title = title_lst[j].text
            company_dct.setdefault(company, {})
            company_dct[company].setdefault(title, 0)
            company_dct[company][title] += 1
            
    return company_dct


def dct2df(dct):
    final_dct = {}
    idx = 0
    
    for company, title_dct in dct.items():
        for title, ct in title_dct.items():
            final_dct[idx] = {'Company':company, 'Title': title, 'Count': ct}
            idx += 1
    df = pd.DataFrame(final_dct).T
    return df

In [47]:
sv_url_pre = "https://www.indeed.com/jobs?q=Data%20Scientist&l=San%20Francisco%2C%20CA&start="
sv_company_dct = get_company_dct(driver, sv_url_pre, pages=20)

In [48]:
sv_company_df = dct2df(sv_company_dct)
sv_company_df.head()

,Company,Count,Title
0,"AI Customized Systems, Inc",14,Chief Data Scientist
1,Stealth Mode Startup,19,Data Analyst
2,openwater (openwater.cc),16,Modeling and Data Analyst Scientist
3,Everlaw,19,Data Operations Associate
4,Astound.ai,4,Field Data Scientist for Machine Learning


In [49]:
vancouver_url_pre = "https://ca.indeed.com/jobs?q=Data+Scientist&l=Vancouver,+BC&start="
vancouver_company_dct = get_company_dct(driver, vancouver_url_pre, pages=7)

In [50]:
vancouver_company_df = dct2df(vancouver_company_dct)
vancouver_company_df.head()

,Company,Count,Title
0,AppleSox,7,Data Scientist
1,Genome Canada Transplant Consortium,4,Data Scientists and Statisticians
2,Motion Metrics International Corp.,7,Machine Learning Developer
3,Ecofish Research Ltd.,7,Senior Fisheries Scientist
4,CARO Analytical Services,3,Inorganics Team Lead


In [51]:
common_trend = vancouver_company_df.merge(sv_company_df, on='Company')
common_trend

,Company,Count_x,Title_x,Count_y,Title_y
0,Electronic Arts,1,Data Engineer,1,Data Scientist
1,Electronic Arts,1,Data Engineer,1,Principal Data Scientist
2,Electronic Arts,1,Data Engineer,1,Research Data Scientist


In [52]:
driver.close()